In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
dircs=os.listdir('/kaggle/input/fishdataset/combine/FishDisease/')

In [ ]:
X_directory=[]
y_directory=[]
for directory in dircs:
    image_paths=os.listdir(f'/kaggle/input/fishdataset/combine/FishDisease/{directory}')
    for img_name in image_paths:
        X_directory.append(f'/kaggle/input/fishdataset/combine/FishDisease/{directory}/{img_name}')
        y_directory.append(directory)

In [ ]:
df={"fishimages":X_directory,"fishlabels":y_directory}

In [ ]:
df = pd.DataFrame(df)
df.style

In [ ]:
import pandas as pd
import numpy as np
df=pd.DataFrame(df)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
df = df.sample(frac=1).reset_index(drop=True)
label_encoder = LabelEncoder()
onehot=OneHotEncoder()
lab=onehot.fit_transform(df['fishlabels'].values.reshape(-1,1)).toarray()
images=df['fishimages']
print('total labels of images',len(lab))
print('total images',len(images))
LAB2=onehot.inverse_transform((lab))
print(lab)
print(LAB2.ravel())    

In [ ]:
import cv2
import matplotlib.pyplot as plt
imgs=[]
for img in images:
    if img.startswith("/kaggle/input/fishdataset/combine/FishDisease/white spot/"):
        other_imag = cv2.imread(img)

        other_frame = cv2.resize(other_imag,(224,224))
        kernel = np.array([[0, -1, 0],
               [-1, 5,-1],
               [0, -1, 0]])
        image_sharp = cv2.filter2D(src=other_frame, ddepth=-1, kernel=kernel)
        hsv = cv2.cvtColor(image_sharp, cv2.COLOR_BGR2HSV)

        # define range of white color in HSV
        # change it according to your need !
        lower_white = np.array([0,0,168], dtype=np.uint8)
        upper_white = np.array([172,111,255],dtype=np.uint8)

        # Threshold the HSV image to get only white colors
        mask = cv2.inRange(hsv, lower_white, upper_white)
        # Bitwise-AND mask and original image
        res = cv2.bitwise_and(image_sharp,image_sharp, mask= mask)
        imgs.append(res)
         
    elif img.startswith("/kaggle/input/fishdataset/combine/FishDisease/red spot/"):
        imag = cv2.imread(img)

        frame = cv2.resize(imag,(224,224)) 

        kernel = np.array([[0, -1, 0],
                   [-1, 5,-1],
                   [0, -1, 0]])
        image_sharp = cv2.filter2D(src=frame, ddepth=-1, kernel=kernel)
        hsv = cv2.cvtColor(image_sharp,cv2.COLOR_BGR2HSV)

        lower_red = np.array([0,50,50])
        upper_red = np.array([10,255,255])


        #upper red
        lower_red2 = np.array([170,50,50])
        upper_red2 = np.array([180,255,255])

        mask = cv2.inRange(hsv, lower_red, upper_red)
        res = cv2.bitwise_and(image_sharp,image_sharp, mask= mask)


        mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
        res2 = cv2.bitwise_and(image_sharp,image_sharp, mask= mask2)

        red=res+res2
        imgs.append(red)
    elif img.startswith("/kaggle/input/fishdataset/combine/FishDisease/black spot/"):
        other_imag = cv2.imread(img)

        other_frame = cv2.resize(other_imag,(224,224))
        kernel = np.array([[0, -1, 0],
               [-1, 5,-1],
               [0, -1, 0]])
        image_sharp = cv2.filter2D(src=other_frame, ddepth=-1, kernel=kernel)
        imagehsv = cv2.cvtColor(image_sharp, cv2.COLOR_BGR2HSV)


        lower_black = np.array([0,0,0])
        upper_black = np.array([179,255,127])

        imagemask = cv2.inRange(imagehsv, lower_black, upper_black)
        result = cv2.bitwise_not(image_sharp,image_sharp, mask=imagemask)

        imgs.append(result)
    elif img.startswith("/kaggle/input/fishdataset/combine/FishDisease/fresh fish/"):
        other_imag = cv2.imread(img)

        other_frame = cv2.resize(other_imag,(224,224))
        kernel = np.array([[0, -1, 0],
               [-1, 5,-1],
               [0, -1, 0]])
        image_sharp = cv2.filter2D(src=other_frame, ddepth=-1, kernel=kernel)

        imgs.append(image_sharp)

In [ ]:
X_preprocess=np.array(imgs)
y_preprocess=np.array(lab)

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns


import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix


# DEEP LEARNING IMPORTS
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv2D, Activation, Dropout, Flatten, MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:

train_x, test_x, train_y, test_y = train_test_split(X_preprocess, y_preprocess, random_state = 42,
                                                   test_size=0.20)

In [ ]:
train_x.shape

In [ ]:
test_x.shape

In [ ]:
X_train_norm=train_x/255.0
X_test_norm=test_x/255.0

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
from math import floor
from sklearn.metrics import make_scorer, accuracy_score
from bayes_opt import BayesianOptimization
from sklearn.model_selection import StratifiedKFold
from keras.layers import LeakyReLU
LeakyReLU = LeakyReLU(alpha=0.1)
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

In [ ]:
def build_model(hp):
    model = keras.Sequential()
    model.add(Conv2D(16,3,3, padding='same',input_shape=(224,224,3), activation=hp.Choice("activation",["relu","tanh"])))

    model.add(MaxPooling2D(pool_size=(2,2),strides=2, padding='same'))
    model.add(Conv2D(256,5,5, padding='same', activation=hp.Choice("activation",["relu","tanh"])))
    model.add(MaxPooling2D(pool_size=(2,2),strides=2, padding='same'))
    model.add(layers.Flatten())
    model.add(
        layers.Dense(
            units=448,
            activation=hp.Choice("activation",["relu","tanh"]),
        )
    )
    model.add(layers.Dense(4, activation="softmax"))
    #compilation of model
    
#     opt = Adam(learning_rate=0.0001)

    model.compile(optimizer='adam',
                      loss="categorical_crossentropy",metrics=['accuracy'])
    return model

In [ ]:
# def build_model(hp):
#     model = keras.Sequential()
#     model.add(Conv2D(16,3,3, padding='same',input_shape=(224,224,3), activation='relu'))

#     model.add(MaxPooling2D(pool_size=(2,2),strides=2, padding='same'))
#     model.add(Conv2D(256,5,5, padding='same', activation='relu'))
#     model.add(MaxPooling2D(pool_size=(2,2),strides=2, padding='same'))
#     model.add(layers.Flatten())
#     model.add(
#         layers.Dense(
#             units=448,
#             activation=hp.Choice("activation",["relu","tanh"]),
#         )
#     )
#     model.add(layers.Dense(4, activation="softmax"))
#     #compilation of model
    
# #     opt = Adam(learning_rate=0.0001)

#     model.compile(optimizer='adam',
#                       loss="categorical_crossentropy",metrics=['accuracy'])
#     return model

**Activation function**
activation: relu
Score: 0.9556786885261536

activation: tanh
Score: 0.9494584798812866

**Optimizers**

In [ ]:
# def build_model(hp):
#     model = keras.Sequential()
#     model.add(Conv2D(16,3,3, padding='same',input_shape=(224,224,3), activation='relu'))

#     model.add(MaxPooling2D(pool_size=(2,2),strides=2, padding='same'))
#     model.add(Conv2D(256,5,5, padding='same', activation='relu'))
#     model.add(MaxPooling2D(pool_size=(2,2),strides=2, padding='same'))
#     model.add(layers.Flatten())
#     model.add(
#         layers.Dense(
#             units=448,
#             activation="relu",
#         )
#     )
#     model.add(layers.Dense(4, activation="softmax"))
#     #compilation of model
    
# #     opt = Adam(learning_rate=lr)

#     model.compile(optimizer=hp.Choice('optimizers',['adam','sgd','RMSprop']),
#                       loss='categorical_crossentropy',metrics=['accuracy'])
#     return model

optimizers: adam
Score: 0.9711191058158875
optimizers: RMSprop
Score: 0.9531187438158485
optimizers: sgd
Score: 0.8375450968742371

In [ ]:
tuner = RandomSearch(build_model,
                    objective='val_accuracy',
                    max_trials = 5)
# search best parameter
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)
tuner.search(X_train_norm, train_y, epochs=30,batch_size=20, validation_data=(X_test_norm, test_y),callbacks=[es])

**LEARNING RATE**

In [ ]:

# def build_model(hp):
#     model = keras.Sequential()
#     model.add(Conv2D(16,3,3, padding='same',input_shape=(224,224,3), activation='relu'))

#     model.add(MaxPooling2D(pool_size=(2,2),strides=2, padding='same'))
#     model.add(Conv2D(256,5,5, padding='same', activation='relu'))
#     model.add(MaxPooling2D(pool_size=(2,2),strides=2, padding='same'))
#     model.add(layers.Flatten())
#     model.add(
#         layers.Dense(
#             units=448,
#             activation="relu",
#         )
#     )
#     model.add(layers.Dense(4, activation="softmax"))
#     #compilation of model
#     lr = hp.Choice("learning_rate",
#     values=[0.00001,0.000001])
#     opt = Adam(learning_rate=lr)

#     model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])
#     return model

learning_rate: 1e-06/0.000001
Score: 0.6462093591690063
learning_rate: 1e-05/0.00001
Score: 0.916967511177063
learning_rate: 0.0001
Score: 0.9655595827102661
learning_rate: 0.001
Score: 0.9438588971710205
learning_rate: 0.01
Score: 0.9144988271410381

In [ ]:
tuner.results_summary()

In [ ]:
# def build_model(hp):
#     model = keras.Sequential()
#     model.add(Conv2D(16,3,3, padding='same',input_shape=(224,224,3), activation='relu'))

#     model.add(MaxPooling2D(pool_size=(2,2),strides=2, padding='same'))
#     model.add(Conv2D(256,5,5, padding='same', activation='relu'))
#     model.add(MaxPooling2D(pool_size=(2,2),strides=2, padding='same'))
#     model.add(layers.Flatten())
#     model.add(
#         layers.Dense(
#             units=448,
#             activation="relu",
#         )
#     )
#     model.add(layers.Dense(4, activation="softmax"))
#     #compilation of model
    
# #     opt = Adam(learning_rate=0.0001)

#     model.compile(optimizer='adam',
#                       loss=hp.Choice("loss",["categorical_crossentropy","binary_crossentropy"]),metrics=['accuracy'])
#     return model

loss: categorical_crossentropy
Score: 0.9638988971710205
loss: binary_crossentropy
Score: 0.9314079284667969

**For epochs and Batch size**

In [ ]:
# import keras_tuner as kt
# class MyHyperModel(kt.HyperModel):
#     def build(self, hp):
#         model = keras.Sequential()
#         model.add(Conv2D(16,3,3, padding='same',input_shape=(224,224,3), activation='relu'))

#         model.add(MaxPooling2D(pool_size=(2,2),strides=2, padding='same'))
#         model.add(Conv2D(256,5,5, padding='same', activation='relu'))
#         model.add(MaxPooling2D(pool_size=(2,2),strides=2, padding='same'))
#         model.add(layers.Flatten())
#         model.add(
#             layers.Dense(
#                 units=448,
#                 activation="relu",
#             )
#         )
#         model.add(layers.Dense(4, activation="softmax"))
#         model.compile(
#             optimizer=keras.optimizers.Adam(0.0001), loss="categorical_crossentropy", metrics=["accuracy"],
#         )
#         return model

#     def fit(self, hp, model, *args, **kwargs):
#         return model.fit(
#             *args,
#             epochs=hp.Choice("epochs", [50]),
#             batch_size=20,
# #           batch_size=hp.Choice("batch_size", [32]),
#             **kwargs,
#         )

# tuner = kt.RandomSearch(
#     MyHyperModel(),
#     objective="val_accuracy",
#     max_trials=3,
#     overwrite=True,
#     directory="my_dir",
#     project_name="tune_hypermodel",
# )

In [ ]:
# import tensorflow
# tuner.search(X_train_norm,train_y,validation_data=(X_test_norm, test_y), callbacks=[tensorflow.keras.callbacks.EarlyStopping('val_loss', patience=3)])

In [ ]:
# class MyHyperModel(kt.HyperModel):
#     def build(self, hp):
#         model = keras.Sequential()
        
#         model.add(layers.Flatten(input_shape=(224,224,3)))
#         model.add(
#             layers.Dense(
#                 units=448,
#                 activation="relu",
#             )
#         )
#         model.add(layers.Dense(4, activation="softmax"))
#         model.compile(
#             optimizer=keras.optimizers.Adam(0.0001), loss="categorical_crossentropy", metrics=["accuracy"],
#         )
#         return model
        

#     def fit(self, hp, model, *args, **kwargs):
#         return model.fit(
#             *args,
#             epochs=hp.Choice("epochs", [50])
#             batch_size=hp.Choice("batch_size", [64]),
#             **kwargs,
#         )

# tuner = kt.RandomSearch(
#     MyHyperModel(),
#     objective="val_accuracy",
#     max_trials=3,
#     overwrite=True,
#     directory="my_dir",
#     project_name="tune_hypermodel",
# )

In [ ]:
tuner.results_summary()

**Epochs**
epochs: 10
Score: 0.9241877198219299
epochs: 20
Score: 0.9494584798812866
epochs: 30
Score: 0.9566786885261536
epochs: 40
Score: 0.9458483457565308
epochs: 50
Score: 0.9386281371116638

**Batch_size**
batch_size: 64
Score: 0.9494584798812866
batch_size: 32
Score: 0.9338988971710205
batch_size: 20
Score: 0.9544787085261432
batch_size: 16
Score: 0.909747302532196
batch_size: 12
Score: 0.943988971710205

In [ ]:
# import keras_tuner as kt
# class MyHyperModel(kt.HyperModel):
#     def build(self, hp):
#         model = keras.Sequential()
#         model.add(Conv2D(16,3,3, padding='same',input_shape=(224,224,3), activation='relu'))
#         model.add(
#             Dropout(rate=hp.Choice(
#                 'dropout',
#                 values=[0.4,0.5],
                
#             )))  
#         model.add(MaxPooling2D(pool_size=(2,2),strides=2, padding='same'))
#         model.add(Conv2D(256,5,5, padding='same', activation='relu'))
#         model.add(MaxPooling2D(pool_size=(2,2),strides=2, padding='same'))
#         model.add(
#                 Dropout(rate=hp.Choice(
#                 'dropout',
#                 values=[0.4,0.5],
                
#             )))  
#         model.add(layers.Flatten())
#         model.add(
#             layers.Dense(
#                 units=448,
#                 activation="relu",
#             )
#         )
#         model.add(
#             Dropout(rate=hp.Choice(
#                 'dropout',
#                 values=[0.1,0.2,0.3],
                
#             ))) 
#         model.add(layers.Dense(4, activation="softmax"))
#         model.compile(
#             optimizer=keras.optimizers.Adam(0.0001), loss="categorical_crossentropy", metrics=["accuracy"],
#         )
#         return model

#     def fit(self, hp, model, *args, **kwargs):
#         return model.fit(
#             *args,
#             epochs=30,
#             batch_size=20,
# #           batch_size=hp.Choice("batch_size", [32]),
#             **kwargs,
#         )

# tuner = kt.RandomSearch(
#     MyHyperModel(),
#     objective="val_accuracy",
#     max_trials=3,
#     overwrite=True,
#     directory="my_dir",
#     project_name="tune_hypermodel",
# )

**Dropout**
dropout: 0.1
Score: 0.9530686140060425
Trial summary
Hyperparameters:
dropout: 0.2
Score: 0.9494584798812866
Trial summary
Hyperparameters:
dropout: 0.3
Score: 0.9205776453018188
dropout: 0.4
Score: 0.9314079284667969
dropout: 0.5
Score: 0.9484583798812361